In [31]:
import requests
from bs4 import BeautifulSoup
import re
import os
import os.path
import base64
import urllib.parse

In [52]:
cache_dir = os.path.expanduser("cache/")
def retrieve_and_cache(url):
    try:
        os.makedirs(cache_dir)
    except:
        print("Cache dir exists")
    filename = urllib.parse.quote(url, '')
    filepath = os.path.join(cache_dir, filename)
    print("Checking cache:", filepath)
    raw_html = ""
    if os.path.exists(filepath):
        print("Retrieving from cache")
        with open(filepath, 'r') as file:
            raw_html = file.read()
    else:
        print("Fetching and saving to cache")
        response = requests.get(url)
        raw_html = response.text
        with open(filepath, 'x') as file:
            file.write(raw_html)
    return raw_html

In [53]:
root_url = "https://www.motorcyclespecs.co.za/Manufacturer.htm"
manufacturers_html = retrieve_and_cache(root_url)

Cache dir exists
Checking cache: cache/https%3A%2F%2Fwww.motorcyclespecs.co.za%2FManufacturer.htm
Retrieving from cache


In [54]:
manufacturers_html

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" itemscope="" itemtype="http://schema.org/Article">\n\n<!-- #BeginTemplate "head.dwt" -->\n\n<head>\n                              <style>\n\n\n                        .Customized_Leaderboard { width: 320px; height: 100px; }\n                        @media(min-width: 500px) { .Customized_Leaderboard { width: 320px; height: 100px; } }\n                        @media(min-width: 801px) { .Customized_Leaderboard { width: 728px; height: 90px; } }\n                        @media(min-width: 1025px) { .Customized_Leaderboard { width: 970px; height: 250px; } }\n                       </style>\n\n                      <style>\n\n\n                         .Billboard { width: 320px; height: 100px; }\n                        @media(min-width: 500px) { .Billboard { width: 320px; height: 100px; } }\n                        @me